In [6]:
import googleapiclient.discovery
import pandas as pd

dev = "YOUR_YOUTUBE_DATA_API_HERE"
api_service_name = "youtube"
api_version = "v3"
DEVELOPER_KEY = dev

youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey=DEVELOPER_KEY)


def getcomments(video):
  request = youtube.commentThreads().list(
      part="snippet",
      videoId=video,
      maxResults=100
  )

  comments = []

  # Execute the request.
  response = request.execute()

  # Get the comments from the response.
  for item in response['items']:
      comment = item['snippet']['topLevelComment']['snippet']
      public = item['snippet']['isPublic']
      comments.append([
          comment['authorDisplayName'],
          comment['publishedAt'],
          comment['likeCount'],
          comment['textOriginal'],
          comment['videoId'],
          public
      ])

  while (1 == 1):
    try:
     nextPageToken = response['nextPageToken']
    except KeyError:
     break
    nextPageToken = response['nextPageToken']
    # Create a new request object with the next page token.
    nextRequest = youtube.commentThreads().list(part="snippet", videoId=video, maxResults=100, pageToken=nextPageToken)
    # Execute the next request.
    response = nextRequest.execute()
    # Get the comments from the next response.
    for item in response['items']:
      comment = item['snippet']['topLevelComment']['snippet']
      public = item['snippet']['isPublic']
      comments.append([
          comment['authorDisplayName'],
          comment['publishedAt'],
          comment['likeCount'],
          comment['textOriginal'],
          comment['videoId'],
          public
      ])

  df2 = pd.DataFrame(comments, columns=['author', 'updated_at', 'like_count', 'text','video_id','public'])
  return df2

In [9]:
from pytube import Playlist

# Replace with your playlist URL
playlist_url = 'https://www.youtube.com/playlist?list=PLCzaIJYXP5Yf57bld-sUTSD2-aCEejZqb'

# Create a Playlist object
playlist = Playlist(playlist_url)

urls = list(playlist.video_urls)
ids = [x.removeprefix('https://www.youtube.com/watch?v=') for x in urls]
print(ids)


['enHai1y5Knc', 'QPAbml1pQ0Q', 'FS3L1k6lHXQ', 'XHDI47WFqe0', 'UPiFHzrWEgM', '1r6yLUKc8yc', 'nrF8gJUB60E', 'QYDf_NQEGUQ', 'bThgzcbe_-Q', 'Tr-TVt5JAWY', 'q27ZhmS6xMY', 'j9GohkxkjGk', '4ZuLUMWdtGI', 'CCWhTwyx-rc', '32zmAixXlZ0', 'k1me0PxMHug', 'O2LKzAdbgeA', 'kS2ydDhLaVw', 'bnah8aIb7CU', 'rXRN_8Wz5kk', 'qYqS5Ib7TxA', 'lol8Gv1g-uA', '_dn3yUfNZKU']


In [10]:
df = pd.DataFrame()
total = len(ids)
for i, id in enumerate(ids):
  df2 = getcomments(id)
  df = pd.concat([df, df2])
  print(f'{i}/{total}')
df.to_csv('comments.csv', index=False)

0/23
1/23
2/23
3/23
4/23
5/23
6/23
7/23
8/23
9/23
10/23
11/23
12/23
13/23
14/23
15/23
16/23
17/23
18/23
19/23
20/23
21/23
22/23


In [12]:
df.to_csv('comments.csv', index=False)